# Preprocessing and Feature Engineering

One of the biggest challenges and perhaps most time consuming step in any advanced analytics project is preprocessing. In order to obtain a meaningful result from a machine learning algorithm a thorough preprocessing is required. It’s not that it’s particularly complicated programming, but rather that it requires deep knowledge of the data you are working with and an understanding of what your model needs in order to successfully leverage this data.

Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. Feature engineering is fundamental to the application of machine learning, and is both difficult and expensive.

## Formatting Models According to Your Use Case

To preprocess data for Spark’s different advanced analytics tools, you must consider your end objective. The following list walks through the requirements for input data structure for each advanced analytics task in MLlib:

* In the case of most **classification** and **regression** algorithms, you want to get your data into a column of type Double to represent the label and a column of type Vector (either dense or sparse) to represent the features.

* In the case of **recommendation**, you want to get your data into a column of users, a column of items (say movies or books), and a column of ratings.

* In the case of **unsupervised learning**, a column of type Vector (either dense or sparse) is needed to represent the features.


Before we proceed, we’re going to read in several different sample datasets, each of which has different properties we will manipulate:

In [1]:
# the following line gets the bucket name attached to our cluster
bucket = spark._jsc.hadoopConfiguration().get("fs.gs.system.bucket")

# specifying the path to our bucket where the data is located (no need to edit this path anymore)
data = "gs://" + bucket + "/Big-Data-Analytics-for-Business/data/"
print(data)

gs://qst843/Big-Data-Analytics-for-Business/data/


### Why we reset Spark’s resource settings

After a recent Spark upgrade, the default configuration may request **very large executor memory** and **many executors** when this notebook runs. On a small single-node Dataproc cluster, this can cause YARN to show a very large **“Pending Memory”** value and the notebook may appear to stall.

If you notice the import cell hanging and the Dataproc *Monitoring → YARN* panel shows unusually high pending memory, simply interrupt the import cell and restart the notebook kernel.

Then, run the cell below to start Spark with **smaller, controlled resource settings**. This ensures the cluster stays responsive.

In [2]:
# Recreate a small, single-node-friendly SparkSession on YARN
try:
    spark.stop()
except Exception:
    pass

from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("single-node-notebook")
    .master("yarn")
    .config("spark.submit.deployMode", "client")
    .config("spark.yarn.unmanagedAM.enabled", "false")   # avoid AM cache/register errors
    .config("spark.dynamicAllocation.enabled", "false")   # stop huge pending asks
    .config("spark.executor.instances", "1")
    .config("spark.executor.cores", "1")
    .config("spark.executor.memory", "2g")
    .config("spark.executor.memoryOverhead", "384m")
    .config("spark.driver.memory", "2g")
    .config("spark.sql.shuffle.partitions", "4")
    .getOrCreate()
)

25/11/09 18:06:18 INFO SparkEnv: Registering MapOutputTracker
25/11/09 18:06:18 INFO SparkEnv: Registering BlockManagerMaster
25/11/09 18:06:18 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
25/11/09 18:06:18 INFO SparkEnv: Registering OutputCommitCoordinator


In [2]:
sales = spark.read.format("csv")\
  .option("header", "true")\
  .option("inferSchema", "true")\
  .load(data + "retail-data/by-day/*.csv")\
  .coalesce(5)\
  .where("Description IS NOT NULL")

fakeIntDF = spark.read.parquet(data + "simple-ml-integers")
simpleDF = spark.read.json(data + "simple-ml")
scaleDF = spark.read.parquet(data + "simple-ml-scaling")

In addition to this realistic *sales* data, we’re going to use several simple synthetic datasets as well. *FakeIntDF*, *simpleDF*, and *scaleDF* all have very few rows. This will give you the ability to focus on the exact data manipulation we are performing instead of the various inconsistencies of any particular dataset. Because we’re going to be accessing the sales data a number of times, we’re going to cache it so we can read it efficiently from memory as opposed to reading it from disk every time we need it. Let’s also check out the first several rows of data in order to better understand what’s in the dataset:

In [3]:
sales.cache()
sales.show(3)
print("sales datasets consists of {} rows.".format(sales.count()))

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   580538|    23084|  RABBIT NIGHT LIGHT|      48|2011-12-05 08:38:00|     1.79|   14075.0|United Kingdom|
|   580538|    23077| DOUGHNUT LIP GLOSS |      20|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|
|   580538|    22906|12 MESSAGE CARDS ...|      24|2011-12-05 08:38:00|     1.65|   14075.0|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
only showing top 3 rows

sales datasets consists of 540455 rows.


**Note:** It is important to note that we filtered out null values here. MLlib does not always play nicely with null values at this point in time. This is a frequent cause for problems and errors and a great first step when you are debugging. Improvements are also made with every Spark release to improve algorithm handling of null values.

## Transformers

The best way to get your data in the desired format is through **transformers**. Transformers are functions that accept a DataFrame as an argument and return a new DataFrame as a response. This notebook will focus on what transformers are relevant for particular use cases rather than attempting to enumerate every possible transformer.

Spark provides a number of transformers that can be found under *pyspark.ml.feature*. For more information on transformers you can visit [Extracting, transforming and selecting features doccumentation page](http://spark.apache.org/docs/latest/ml-features.html).

Transformers are functions that convert raw data in some way. This might be to create a new interaction variable (from two other variables), to normalize a column, or to simply turn it into a Double to be input into a model. Transformers are primarily used in preprocessing or feature generation. RFormula that we saw in the previous lecture is an example of a transformer.

Spark’s transformer only includes a transform method. This is because it will not change based on the input data. Figure below is a simple illustration. On the left is an input DataFrame with the column to be manipulated. On the right is the input DataFrame with a new column representing the output transformation.

<img src="https://github.com/soltaniehha/Big-Data-Analytics-for-Business/blob/master/figs/10-01-A-Spark-transformer.png?raw=true" width="800" align="center"/>

The **`Tokenizer`** is an example of a transformer. It tokenizes a string (converts the input string to lowercase and then splits it by white spaces) and has nothing to learn from our data; it simply applies a function. Here’s a small code snippet showing how a tokenizer is built to accept the input column, how it transforms the data, and then the output from that transformation:

In [4]:
from pyspark.ml.feature import Tokenizer

tokenizer = Tokenizer(inputCol="Description", outputCol="words")
tokenized = tokenizer.transform(sales).select("Description", "words")
tokenized.show(5, False)

+-------------------------------+-------------------------------------+
|Description                    |words                                |
+-------------------------------+-------------------------------------+
|RABBIT NIGHT LIGHT             |[rabbit, night, light]               |
|DOUGHNUT LIP GLOSS             |[doughnut, lip, gloss]               |
|12 MESSAGE CARDS WITH ENVELOPES|[12, message, cards, with, envelopes]|
|BLUE HARMONICA IN BOX          |[blue, harmonica, in, box]           |
|GUMBALL COAT RACK              |[gumball, coat, rack]                |
+-------------------------------+-------------------------------------+
only showing top 5 rows



We can simply create a "udf" function and apply it on our newly created column to count the number of words:

In [5]:
from pyspark.sql import functions as F

countTokens = F.udf(lambda words: len(words))

tokenized.select("Description", "words")\
    .withColumn("tokens", countTokens(F.col("words"))).show(5, False)

+-------------------------------+-------------------------------------+------+
|Description                    |words                                |tokens|
+-------------------------------+-------------------------------------+------+
|RABBIT NIGHT LIGHT             |[rabbit, night, light]               |3     |
|DOUGHNUT LIP GLOSS             |[doughnut, lip, gloss]               |3     |
|12 MESSAGE CARDS WITH ENVELOPES|[12, message, cards, with, envelopes]|5     |
|BLUE HARMONICA IN BOX          |[blue, harmonica, in, box]           |4     |
|GUMBALL COAT RACK              |[gumball, coat, rack]                |3     |
+-------------------------------+-------------------------------------+------+
only showing top 5 rows



## Estimators for Preprocessing

Another set of tools for preprocessing are estimators. An estimator is necessary when a transformation you would like to perform must be initialized with data or information about the input column (often derived by doing a pass over the input column itself). For example, if you wanted to scale the values in our column to have mean zero and unit variance, you would need to perform a pass over the entire data in order to calculate the values you would use to normalize the data to mean zero and unit variance. In effect, an estimator can be a transformer configured according to your particular input data. In simplest terms, you can either blindly apply a transformation (a “regular” transformer type) or perform a transformation based on your data (an estimator type). Figure below is a simple illustration of an estimator fitting to a particular input dataset, generating a transformer that is then applied to the input dataset to append a new column (of the transformed data).

<img src="https://github.com/soltaniehha/Big-Data-Analytics-for-Business/blob/master/figs/10-01-A-Spark-estimator.png?raw=true" width="800" align="center"/>


An example of this type of estimator is the **`StandardScaler`**, which scales your input column according to the range of values in that column to have a zero mean and a variance of 1 in each dimension. For that reason it must first perform a pass over the data to create the transformer. Here’s a sample code snippet showing the entire process, as well as the output:

In [6]:
scaleDF.show(2)

+---+--------------+
| id|      features|
+---+--------------+
|  0|[1.0,0.1,-1.0]|
|  1| [2.0,1.1,1.0]|
+---+--------------+
only showing top 2 rows



In [7]:
from pyspark.ml.feature import StandardScaler

ss = StandardScaler(inputCol="features", outputCol="scaledFeatures")

FittedSs = ss.fit(scaleDF)
print("mean:", FittedSs.mean)
print("std:", FittedSs.std)

mean: [1.8,2.5,0.6]
std: [0.8366600265340756,4.277849927241488,1.6733200530681511]


In [8]:
FittedSs.transform(scaleDF).show(5, False)

+---+--------------+------------------------------------------------------------+
|id |features      |scaledFeatures                                              |
+---+--------------+------------------------------------------------------------+
|0  |[1.0,0.1,-1.0]|[1.1952286093343936,0.02337622911060922,-0.5976143046671968]|
|1  |[2.0,1.1,1.0] |[2.390457218668787,0.2571385202167014,0.5976143046671968]   |
|0  |[1.0,0.1,-1.0]|[1.1952286093343936,0.02337622911060922,-0.5976143046671968]|
|1  |[2.0,1.1,1.0] |[2.390457218668787,0.2571385202167014,0.5976143046671968]   |
|1  |[3.0,10.1,3.0]|[3.5856858280031805,2.3609991401715313,1.7928429140015902]  |
+---+--------------+------------------------------------------------------------+



**Note** that we have used the same DF to fit and to transform our data, but in reality one could use a different DF to obtain the parameters needed to scale (e.g. on the training set) and then apply it to other DFs (test, validation).

### Transformer Properties

All transformers require you to specify, at a minimum, the inputCol and the outputCol, which represent the column name of the input and output, respectively. There are some defaults (you can find these in the documentation), but it is a best practice to manually specify them yourself for clarity. In addition to input and output columns, all transformers have different parameters that you can tune (whenever we mention a parameter in this chapter you must set it with a set() method). In Python, we also have another method to set these values with keyword arguments to the object’s constructor. Estimators require you to fit the transformer to your particular dataset and then call transform on the resulting object.

## High-Level Transformers

High-level transformers, such as the RFormula we saw in the previous lecture, allow you to concisely specify a number of transformations in one. These operate at a “high level”, and allow you to avoid doing data manipulations or transformations one by one. In general, **you should try to use the highest level transformers you can, in order to minimize the risk of error and help you focus on the business problem instead of the smaller details of implementation**. While this is not always possible, it’s a good objective.

### RFormula

The **`RFormula`** is the easiest transfomer to use when you have “conventionally” formatted data. Spark borrows this transformer from the R language to make it simple to declaratively specify a set of transformations for your data. With this transformer, values can be either numerical or categorical and you do not need to extract values from strings or manipulate them in any way. The RFormula will automatically handle categorical inputs (specified as strings) by performing something called one-hot encoding. In brief, one-hot encoding converts a set of values into a set of binary columns specifying whether or not the data point has each particular value (we’ll discuss one-hot encoding in more depth later in the notebook). With the RFormula, numeric columns will be cast to Double but will not be one-hot encoded. If the label column is of type String, it will be first transformed to Double with StringIndexer.

The RFormula allows you to specify your transformations in declarative syntax. It is simple to use once you understand the syntax. Currently, RFormula supports a limited subset of the R operators that in practice work quite well for simple transformations. The basic operators are:

~
Separate target and terms

+
Concat terms; “+ 0” means removing the intercept (this means that the y-intercept of the line that we will fit will be 0)

-
Remove a term; “- 1” means removing the intercept (this means that the y-intercept of the line that we will fit will be 0—yes, this does the same thing as “+ 0”

:
Interaction (multiplication for numeric values, or binarized categorical values)

.
All columns except the target/dependent variable

RFormula also uses default columns of label and features to label, you guessed it, the label and the set of features that it outputs (for supervised machine learning). The models covered later on in this chapter by default require those column names, making it easy to pass the resulting transformed DataFrame into a model for training.

Let’s use RFormula in an example. In this case, we want to use all available variables (the .) and then specify an interaction between value1 and color and value2 and color as additional features to generate:

In [9]:
from pyspark.ml.feature import RFormula

supervised = RFormula(formula="lab ~ . + color:value1 + color:value2")
supervised.fit(simpleDF).transform(simpleDF).show(2, False)

+-----+----+------+------------------+--------------------------------------------------------------------+-----+
|color|lab |value1|value2            |features                                                            |label|
+-----+----+------+------------------+--------------------------------------------------------------------+-----+
|green|good|1     |14.386294994851129|(10,[1,2,3,5,8],[1.0,1.0,14.386294994851129,1.0,14.386294994851129])|1.0  |
|blue |bad |8     |14.386294994851129|(10,[2,3,6,9],[8.0,14.386294994851129,8.0,14.386294994851129])      |0.0  |
+-----+----+------+------------------+--------------------------------------------------------------------+-----+
only showing top 2 rows



### SQL Transformers

A **`SQLTransformer`** allows you to leverage Spark’s vast library of SQL-related manipulations just as you would a MLlib transformation. Any SELECT statement you can use in SQL is a valid transformation. The only thing you need to change is that instead of using the table name, you should just use the keyword \__THIS__. You might want to use SQLTransformer if you want to formally codify some DataFrame manipulation as a preprocessing step, or try different SQL expressions for features during hyperparameter tuning. Also note that the output of this transformation will be appended as a column to the output DataFrame.

You might want to use an SQLTransformer in order to represent all of your manipulations on the very rawest form of your data so you can version different variations of manipulations as transformers. This gives you the benefit of building and testing varying pipelines, all by simply swapping out transformers. The following is a basic example of using SQLTransformer:

In [10]:
from pyspark.ml.feature import SQLTransformer

basicTransformation = SQLTransformer()\
  .setStatement("""
    SELECT CustomerID, sum(Quantity) total_quantity, count(*) number_of_transactions
    FROM __THIS__
    GROUP BY CustomerID
  """)

basicTransformation.transform(sales).show(5)

+----------+--------------+----------------------+
|CustomerID|total_quantity|number_of_transactions|
+----------+--------------+----------------------+
|   14024.0|           150|                    16|
|   17343.0|            56|                    17|
|   13849.0|          1016|                   101|
|   13871.0|          2801|                   338|
|   16550.0|          1526|                   261|
+----------+--------------+----------------------+
only showing top 5 rows



### VectorAssembler

The **`VectorAssembler`** is a tool you’ll use in nearly every single pipeline you generate. It helps concatenate all your features into one big vector you can then pass into an estimator. It’s used typically in the last step of a machine learning pipeline and takes as input a number of columns of Boolean, Double, or Vector. This is particularly helpful if you’re going to perform a number of manipulations using a variety of transformers and need to gather all of those results together.

Its output is similar to an RFormula transformation without the label column and any particular transformation based on a formula.

The output from the following code snippet will make it clear how this works:

In [11]:
from pyspark.ml.feature import VectorAssembler

va = VectorAssembler().setInputCols(["int1", "int2", "int3"])
va.transform(fakeIntDF).show()

+----+----+----+------------------------------------+
|int1|int2|int3|VectorAssembler_df877a721a54__output|
+----+----+----+------------------------------------+
|   1|   2|   3|                       [1.0,2.0,3.0]|
|   4|   5|   6|                       [4.0,5.0,6.0]|
|   7|   8|   9|                       [7.0,8.0,9.0]|
+----+----+----+------------------------------------+



## Working with Continuous Features

Continuous features are just values on the number line, from positive infinity to negative infinity. There are two common transformers for continuous features. First, you can convert continuous features into categorical features via a process called bucketing, or you can scale and normalize your features according to several different requirements. These transformers will only work on Double types, so make sure you’ve turned any other numerical values to Double:

In [12]:
contDF = spark.range(20).selectExpr("cast(id as double)")
contDF.show(2)

+---+
| id|
+---+
|0.0|
|1.0|
+---+
only showing top 2 rows



### Bucketing

The most straightforward approach to *bucketing* or *binning* is using the **`Bucketizer`**. This will split a given continuous feature into the buckets of your designation. You specify how buckets should be created via an array or list of Double values. This is useful because you may want to simplify the features in your dataset or simplify their representations for interpretation later on. For example, imagine you have a column that represents a patient’s BMI and you would like to predict some value based on this information. In some cases, it might be simpler to create three buckets of “overweight,” “average,” and “underweight.”

To specify the bucket, set its borders. For example, setting splits to 5.0, 10.0, 250.0 on our contDF will actually fail because we don’t cover all possible input ranges. When specifying your bucket points, the values you pass into splits must satisfy three requirements:

* The minimum value in your splits array must be less than the minimum value in your DataFrame.

* The maximum value in your splits array must be greater than the maximum value in your DataFrame.

* You need to specify at a minimum three values in the splits array, which creates two buckets.

To cover all possible ranges we can use `float("inf")` and `float("-inf")`.

In order to handle null or NaN values, we must specify the `handleInvalid` parameter as a certain value. We can either keep those values (keep), error or null, or skip those rows. Here’s an example of using bucketing:

In [13]:
from pyspark.ml.feature import Bucketizer

bucketBorders = [-1.0, 5.0, 10.0, 250.0, 600.0]
bucketer = Bucketizer(inputCol="id", outputCol="id_bucket", splits = bucketBorders)
bucketer.transform(contDF).show()

+----+---------+
|  id|id_bucket|
+----+---------+
| 0.0|      0.0|
| 1.0|      0.0|
| 2.0|      0.0|
| 3.0|      0.0|
| 4.0|      0.0|
| 5.0|      1.0|
| 6.0|      1.0|
| 7.0|      1.0|
| 8.0|      1.0|
| 9.0|      1.0|
|10.0|      2.0|
|11.0|      2.0|
|12.0|      2.0|
|13.0|      2.0|
|14.0|      2.0|
|15.0|      2.0|
|16.0|      2.0|
|17.0|      2.0|
|18.0|      2.0|
|19.0|      2.0|
+----+---------+



In addition to splitting based on hardcoded values, another option is to split based on percentiles in our data. This is done with **`QuantileDiscretizer`**, which will bucket the values into user-specified buckets with the splits being determined by approximate quantiles values. For instance, the 90th quantile is the point in your data at which 90% of the data is below that value. You can control how finely the buckets should be split by setting the relative error for the approximate quantiles calculation using `setRelativeError`. Spark does this is by allowing you to specify the number of buckets you would like out of the data and it will split up your data accordingly. The following is an example:

In [14]:
from pyspark.ml.feature import QuantileDiscretizer

bucketer = QuantileDiscretizer(numBuckets=5, inputCol="id", outputCol="id_bucket")
fittedBucketer = bucketer.fit(contDF)
fittedBucketer.transform(contDF).show()

+----+---------+
|  id|id_bucket|
+----+---------+
| 0.0|      0.0|
| 1.0|      0.0|
| 2.0|      0.0|
| 3.0|      1.0|
| 4.0|      1.0|
| 5.0|      1.0|
| 6.0|      1.0|
| 7.0|      2.0|
| 8.0|      2.0|
| 9.0|      2.0|
|10.0|      2.0|
|11.0|      3.0|
|12.0|      3.0|
|13.0|      3.0|
|14.0|      3.0|
|15.0|      4.0|
|16.0|      4.0|
|17.0|      4.0|
|18.0|      4.0|
|19.0|      4.0|
+----+---------+



**NOTE:** If the user chooses to keep NaN values, they will be handled specially and placed into their own bucket, for example, if 4 buckets are used, then non-NaN data will be put into buckets[0-3], but NaNs will be counted in a special bucket[4].

### Scaling and Normalization

We saw how we can use bucketing to create groups out of continuous variables. Another common task is to scale and normalize continuous data. While not always necessary, doing so is usually a best practice. You might want to do this when your data contains a number of columns based on different scales. For instance, say we have a DataFrame with two columns: weight (in ounces) and height (in feet). If you don’t scale or normalize, the algorithm will be less sensitive to variations in height because height values in feet are much lower than weight values in ounces. That’s an example where you should scale your data.

An example of normalization might involve transforming the data so that each point’s value is a representation of its distance from the mean of that column. Using the same example from before, we might want to know how far a given individual’s height is from the mean height. Many algorithms assume that their input data is normalized.

As you might imagine, there are a multitude of algorithms we can apply to our data to scale or normalize it. Enumerating them all is unnecessary here because they are covered in many other texts and machine learning libraries. If you’re unfamiliar with the concept in detail, check out any of the books referenced in the previous lecture. Just keep in mind the fundamental goal—we want our data on the same scale so that values can easily be compared to one another in a sensible way. 

**`StandardScaler`**

The `StandardScaler` standardizes a set of features to have zero mean and a standard deviation of 1. The flag `withStd` will scale the data to unit standard deviation while the flag `withMean` (false by default) will center the data prior to scaling it.

WARNING: Centering can be very expensive on sparse vectors because it generally turns them into dense vectors, so be careful before centering your data.

Here’s an example of using a `StandardScaler`:

In [15]:
from pyspark.ml.feature import StandardScaler

sScaler = StandardScaler(inputCol="features", withMean=True)
sScaler.fit(scaleDF).transform(scaleDF).show(5, False)

+---+--------------+--------------------------------------------------------------+
|id |features      |StandardScaler_608004644bb1__output                           |
+---+--------------+--------------------------------------------------------------+
|0  |[1.0,0.1,-1.0]|[-0.9561828874675149,-0.5610294986546213,-0.9561828874675149] |
|1  |[2.0,1.1,1.0] |[0.23904572186687867,-0.32726720754852906,0.23904572186687872]|
|0  |[1.0,0.1,-1.0]|[-0.9561828874675149,-0.5610294986546213,-0.9561828874675149] |
|1  |[2.0,1.1,1.0] |[0.23904572186687867,-0.32726720754852906,0.23904572186687872]|
|1  |[3.0,10.1,3.0]|[1.4342743312012722,1.7765934124063008,1.4342743312012722]    |
+---+--------------+--------------------------------------------------------------+



**`MINMAXSCALER`**

The `MinMaxScaler` will scale the values in a vector (component wise) to the proportional values on a scale from a given min value to a max value. If you specify the minimum value to be 0 and the maximum value to be 1, then all the values will fall in between 0 and 1:

In [16]:
from pyspark.ml.feature import MinMaxScaler

minMax = MinMaxScaler(min=0, max=1, inputCol="features", outputCol="features_MinMaxScaler")
fittedminMax = minMax.fit(scaleDF)
fittedminMax.transform(scaleDF).show()

+---+--------------+---------------------+
| id|      features|features_MinMaxScaler|
+---+--------------+---------------------+
|  0|[1.0,0.1,-1.0]|            (3,[],[])|
|  1| [2.0,1.1,1.0]|        [0.5,0.1,0.5]|
|  0|[1.0,0.1,-1.0]|            (3,[],[])|
|  1| [2.0,1.1,1.0]|        [0.5,0.1,0.5]|
|  1|[3.0,10.1,3.0]|        [1.0,1.0,1.0]|
+---+--------------+---------------------+



**`MAXABSSCALER`**

The max absolute scaler (`MaxAbsScaler`) scales the data by dividing each value by the maximum absolute value in this feature. All values therefore end up between −1 and 1. This transformer does not shift or center the data at all in the process:

In [17]:
from pyspark.ml.feature import MaxAbsScaler
maScaler = MaxAbsScaler(inputCol="features", outputCol="features_MaxAbsScaler")
fittedmaScaler = maScaler.fit(scaleDF)
fittedmaScaler.transform(scaleDF).show(5, False)

+---+--------------+-------------------------------------------------------------+
|id |features      |features_MaxAbsScaler                                        |
+---+--------------+-------------------------------------------------------------+
|0  |[1.0,0.1,-1.0]|[0.3333333333333333,0.009900990099009903,-0.3333333333333333]|
|1  |[2.0,1.1,1.0] |[0.6666666666666666,0.10891089108910892,0.3333333333333333]  |
|0  |[1.0,0.1,-1.0]|[0.3333333333333333,0.009900990099009903,-0.3333333333333333]|
|1  |[2.0,1.1,1.0] |[0.6666666666666666,0.10891089108910892,0.3333333333333333]  |
|1  |[3.0,10.1,3.0]|[1.0,1.0,1.0]                                                |
+---+--------------+-------------------------------------------------------------+



## Working with Categorical Features

The most common task for categorical features is indexing. Indexing converts a categorical variable in a column to a numerical one that you can plug into machine learning algorithms. While this is conceptually simple, there are some catches that are important to keep in mind so that Spark can do this in a stable and repeatable manner.

In general, we recommend re-indexing every categorical variable when pre-processing just for consistency’s sake. This can be helpful in maintaining your models over the long run as your encoding practices may change over time.

**`StringIndexer`** 

The simplest way to index is via the StringIndexer, which maps strings to different numerical IDs. Spark’s StringIndexer also creates metadata attached to the DataFrame that specify what inputs correspond to what outputs. This allows us later to get inputs back from their respective index values:

In [18]:
from pyspark.ml.feature import StringIndexer

lblIndxr = StringIndexer(inputCol="lab", outputCol="labelInd")
idxRes = lblIndxr.fit(simpleDF).transform(simpleDF)
idxRes.select("lab", "labelInd").show(5)

+----+--------+
| lab|labelInd|
+----+--------+
|good|     1.0|
| bad|     0.0|
| bad|     0.0|
|good|     1.0|
|good|     1.0|
+----+--------+
only showing top 5 rows



The default behavior is `stringOrderType='frequencyDesc'`, meaning that the most frequent label gets index 0.

We can also apply `StringIndexer` to columns that are not strings, in which case, they will be converted to strings before being indexed:

In [19]:
valIndexer = StringIndexer(inputCol="value1", outputCol="value1Ind")
valIndexer.fit(simpleDF).transform(simpleDF).select("value1", "value1Ind").show(10)

+------+---------+
|value1|value1Ind|
+------+---------+
|     1|      0.0|
|     8|      7.0|
|    12|      1.0|
|    15|      3.0|
|    12|      1.0|
|    16|      2.0|
|    35|      5.0|
|     1|      0.0|
|     2|      4.0|
|    16|      2.0|
+------+---------+
only showing top 10 rows



Keep in mind that the StringIndexer is an estimator that must be fit on the input data. This means it must see all inputs to select a mapping of inputs to IDs. If you train a StringIndexer on inputs “a,” “b,” and “c” and then go to use it against input “d,” it will throw an error by default. Another option is to skip the entire row if the input value was not a value seen during training. Going along with the previous example, an input value of “d” would cause that row to be skipped entirely. We can set this option before or after training the indexer or pipeline. More options may be added to this feature in the future but as of Spark 2.2, you can only skip or throw an error on invalid inputs.

In [20]:
valIndexer.setHandleInvalid("skip")

StringIndexer_87085ec9a768

Or while definition:

In [21]:
valIndexer = StringIndexer(inputCol="value1", outputCol="value1Ind", handleInvalid="skip")

### Converting Indexed Values Back to Text

When inspecting your machine learning results, you’re likely going to want to map back to the original values. Since MLlib classification models make predictions using the indexed values, this conversion is useful for converting model predictions (indices) back to the original categories. We can do this with `IndexToString`. You’ll notice that we do not have to input our value to the String key; Spark’s MLlib maintains this metadata for you. You can optionally specify the outputs.

In [22]:
from pyspark.ml.feature import IndexToString

labelReverse = IndexToString(inputCol="labelInd")
labelReverse.transform(idxRes).show(5)

+-----+----+------+------------------+--------+----------------------------------+
|color| lab|value1|            value2|labelInd|IndexToString_0060a9dd4794__output|
+-----+----+------+------------------+--------+----------------------------------+
|green|good|     1|14.386294994851129|     1.0|                              good|
| blue| bad|     8|14.386294994851129|     0.0|                               bad|
| blue| bad|    12|14.386294994851129|     0.0|                               bad|
|green|good|    15| 38.97187133755819|     1.0|                              good|
|green|good|    12|14.386294994851129|     1.0|                              good|
+-----+----+------+------------------+--------+----------------------------------+
only showing top 5 rows



**`One-Hot Encoding`**

Indexing categorical variables is only half of the story. One-hot encoding is an extremely common data transformation performed after indexing categorical variables. This is because indexing does not always represent our categorical variables in the correct way for downstream models to process. For instance, when we index our “color” column, you will notice that some colors have a higher value (or index number) than others (in our case, blue is 1 and green is 2).

This is incorrect because it gives the mathematical appearance that the input to the machine learning algorithm seems to specify that green > blue, which makes no sense in the case of the current categories. To avoid this, we use `OneHotEncoder`, which will convert each distinct value to a Boolean flag (1 or 0) as a component in a vector. When we encode the color value, then we can see these are no longer ordered, making them easier for downstream models (e.g., a linear model) to process:

In [23]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer

lblIndxr = StringIndexer(inputCol="color", outputCol="colorInd")
colorLab = lblIndxr.fit(simpleDF).transform(simpleDF.select("color"))
ohe = OneHotEncoder(inputCol="colorInd", outputCol="one-hot-enc-color").fit(colorLab)
ohe.transform(colorLab).show(5)

+-----+--------+-----------------+
|color|colorInd|one-hot-enc-color|
+-----+--------+-----------------+
|green|     1.0|    (2,[1],[1.0])|
| blue|     2.0|        (2,[],[])|
| blue|     2.0|        (2,[],[])|
|green|     1.0|    (2,[1],[1.0])|
|green|     1.0|    (2,[1],[1.0])|
+-----+--------+-----------------+
only showing top 5 rows



### Text Data Transformers

Text is always tricky input because it often requires lots of manipulation to map to a format that a machine learning model will be able to use effectively. There are generally two kinds of texts you’ll see: free-form text and string categorical variables. We already discussed categorical variables in the previous section. At the beginning of this notebook we also saw an example of free-form text and were able to tokenize it. Text is a very interesting and broad discussion and beyond the scope of this course. For more text transformations I refer you to chapter 25 of the textbook and Spark's documentation.

### Further Reading

For more information on transformers visit [Extracting, transforming and selecting features doccumentation page](http://spark.apache.org/docs/latest/ml-features.html).